In [1]:
using Pkg

Pkg.activate(mktempdir())

Pkg.update()

Pkg.add([
    "Flux",
    "Statistics",
])

  Activating new project at `C:\Users\79021\AppData\Local\Temp\jl_hgblHj`
    Updating registry at `C:\Users\79021\.julia\registries\General.toml`
     Project No packages added to or removed from `C:\Users\79021\AppData\Local\Temp\jl_hgblHj\Project.toml`
    Manifest No packages added to or removed from `C:\Users\79021\AppData\Local\Temp\jl_hgblHj\Manifest.toml`
   Resolving package versions...
   Installed Optimisers ──── v0.4.7
   Installed MLDataDevices ─ v1.16.0
   Installed Flux ────────── v0.16.7
    Updating `C:\Users\79021\AppData\Local\Temp\jl_hgblHj\Project.toml`
  [587475ba] + Flux v0.16.7
  [10745b16] + Statistics v1.11.1
    Updating `C:\Users\79021\AppData\Local\Temp\jl_hgblHj\Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [7d9f7c33] + Accessors v0.1.43
  [79e6a3ab] + Adapt v4.4.0
  [66dad0bd] + AliasTables v1.1.3
  [dce04be8] + ArgCheck v2.5.0
  [a9b6321e] + Atomix v1.1.2
  [198e06fe] + BangBang v0.4.6
  [9718e550] + Baselet v0.1.1
  [082447d4] + ChainRules v1.72.6

In [2]:
using Flux, Statistics, Random

# Binary Classification: Predicting disease risk from health metrics
# Input: 3 features (blood pressure, glucose, BMI)
# Output: 0 (healthy) or 1 (at risk)

Random.seed!(42)

# Generate synthetic patient data
n_samples = 1000
X = Float32.(hcat(
    120 .+ 20 * randn(n_samples),      # blood pressure
    90 .+ 15 * randn(n_samples),       # glucose level
    25 .+ 5 * randn(n_samples)         # BMI
)')

# Label: at risk if high BP + high glucose + high BMI
y = Float32.((X[1,:] .> 130) .& (X[2,:] .> 100) .& (X[3,:] .> 28))

# Split train/test
train_idx = 1:800
test_idx = 801:1000

X_train, X_test = X[:, train_idx], X[:, test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# Model: neural network classifier
# Architecture: 3 inputs -> 16 hidden -> 8 hidden -> 1 output (probability)
model = Chain(
    Dense(3 => 16, relu),              # first hidden layer
    Dense(16 => 8, relu),              # second hidden layer
    Dense(8 => 1, σ)                   # output layer with sigmoid activation
)

# Setup optimizer (Adam with learning rate 0.01)
opt_state = Flux.setup(Adam(0.01), model)

# Training loop
println("Training binary classifier...")
for epoch in 1:100
    # Compute gradients
    grads = gradient(model) do m
        ŷ = vec(m(X_train))            # predictions on training data
        Flux.binarycrossentropy(ŷ, y_train)  # loss: binary cross-entropy
    end
    
    # Update model parameters
    Flux.update!(opt_state, model, grads[1])
    
    # Print progress every 20 epochs
    if epoch % 20 == 0
        train_loss = Flux.binarycrossentropy(vec(model(X_train)), y_train)
        println("Epoch $epoch: Loss = $(round(train_loss, digits=4))")
    end
end

# Evaluate on test set
y_pred = vec(model(X_test)) .> 0.5  # threshold at 0.5 for binary decision
accuracy = mean(y_pred .== y_test)
println("Test accuracy: $(round(accuracy * 100, digits=2))%")

Training binary classifier...
Epoch 20: Loss = 0.4384
Epoch 40: Loss = 0.4384
Epoch 60: Loss = 0.4384
Epoch 80: Loss = 0.4384
Epoch 100: Loss = 0.4384
Test accuracy: 98.5%


In [3]:
using Flux, Statistics, Random

# Regression: Predicting house prices
# Input: 3 features (size in sqft, bedrooms, location_score)
# Output: price in thousands of dollars

Random.seed!(123)

# Generate synthetic housing data
n_samples = 800
X = Float32.(hcat(
    1000 .+ 800 * rand(n_samples),     # house size (1000-1800 sqft)
    2 .+ rand([0, 1, 2], n_samples),   # bedrooms (2-4)
    5 .+ 5 * rand(n_samples)           # location score (5-10)
)')

# True price model: base_price + size_effect + bedroom_effect + location_effect + noise
y = reshape(
    100 .+                              # base price 100k
    0.15 * X[1,:] .+                   # +$150 per sqft
    20 * X[2,:] .+                     # +$20k per bedroom
    15 * X[3,:] .+                     # +$15k per location point
    10 * randn(Float32, n_samples),    # noise
    1, :
)

# Split data
train_idx = 1:600
test_idx = 601:800

X_train, X_test = X[:, train_idx], X[:, test_idx]
y_train, y_test = y[:, train_idx], y[:, test_idx]

# Model: neural network regressor
# Architecture: 3 inputs -> 32 hidden -> 16 hidden -> 1 output (price)
model = Chain(
    Dense(3 => 32, tanh),              # first hidden layer with tanh activation
    Dense(32 => 16, tanh),             # second hidden layer
    Dense(16 => 1)                     # output layer (linear, no activation)
)

# Setup optimizer
opt_state = Flux.setup(Adam(0.01), model)

# Training loop
println("\nTraining regression model...")
for epoch in 1:200
    # Compute gradients
    grads = gradient(model) do m
        ŷ = m(X_train)                 # predictions
        Flux.mse(ŷ, y_train)           # loss: mean squared error
    end
    
    # Update parameters
    Flux.update!(opt_state, model, grads[1])
    
    # Print progress
    if epoch % 40 == 0
        train_loss = Flux.mse(model(X_train), y_train)
        println("Epoch $epoch: MSE = $(round(train_loss, digits=2))")
    end
end

# Evaluate on test set
y_pred = model(X_test)
test_mse = Flux.mse(y_pred, y_test)
println("Test MSE: $(round(test_mse, digits=2))")

# Show some predictions vs actual
println("\nSample predictions (first 5 test samples):")
for i in 1:5
    println("Predicted: \$$(round(y_pred[i], digits=1))k, Actual: \$$(round(y_test[i], digits=1))k")
end


Training regression model...
Epoch 40: MSE = 226845.83
Epoch 80: MSE = 220328.65
Epoch 120: MSE = 214015.33
Epoch 160: MSE = 207866.32
Epoch 200: MSE = 201865.29
Test MSE: 197293.06

Sample predictions (first 5 test samples):
Predicted: $38.2k, Actual: $590.2k
Predicted: $38.2k, Actual: $538.2k
Predicted: $38.2k, Actual: $454.4k
Predicted: $38.2k, Actual: $520.2k
Predicted: $38.2k, Actual: $413.1k
